huggingface의 monologg/koelectra-base-v3-discriminator 모델을 사용하여 문제의 클래스를 예측하는 과정이 담긴 코드입니다.

#라이브러리 설치

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.3 MB/s 
     |████████████████████████████████| 596 kB 45.3 MB/s 
     |████████████████████████████████| 6.6 MB 42.4 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# 1- 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

data_path = '/content/drive/MyDrive/KMWP/code/data/oversampled_sep_train.csv'

data = pd.read_csv(data_path)
print(data.shape)
data.head()

(7096, 2)


,problem,class
0,한 변의 길이가 24cm인 정육각형과 둘레가 같은 정팔각형이 있습니다. 이 정팔각형...,7
1,윤아는 부추전을 똑같이 8조각으로 나누어 한 조각을 먹었습니다. 윤미는 같은 크기의...,0
2,"화단 주변에 한 변이 12m인 정팔각형 모양의 울타리를 두른다면, 울타리는 모두 몇...",7
3,"6장의 숫자 카드 0, 9, 8, 7, 2, 1가 있습니다. 이를, 한 번씩 사용하...",2
4,0.26 x 0.8을 계산해 주세요.,0


In [ ]:
data['class'].value_counts()

7    887
0    887
2    887
6    887
5    887
4    887
3    887
1    887
Name: class, dtype: int64

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/KMWP/code/data/sep_test.csv')

print(test_data.shape)
test_data.tail()

(282, 2)


,problem,label
277,냉장고에 우유가 3L 있었습니다. 은채는 오늘 우유 전체의 1/3를 마셨습니다. 은...,0
278,"가로가 6센티미터, 세로가 3센티미터인 직육면체가 있습니다. 이 직육면체의 부피가 ...",7
279,고리에 걸릴 때마다 3점씩 얻고 걸리지 않으 면 2점씩 잃는다고 합니다. 윤지는 고...,0
280,"냉장고에 우유가 4리터 있었습니다. 세은이는 우유 전체의 1/4을 마셨다면, 세은이...",0
281,"넓이가 20센티미터² 인 정육각형을 10등분하였다면, 한 등분의 넓이는 몇 센티미터...",7


In [ ]:
test_data.label.value_counts()

0    113
7     81
3     26
2     17
5     15
6     14
4     14
1      2
Name: label, dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7096 entries, 0 to 7095
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   problem  7096 non-null   object
 1   class    7096 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 111.0+ KB


In [ ]:
data = data.astype({'class':'int32'})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7096 entries, 0 to 7095
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   problem  7096 non-null   object
 1   class    7096 non-null   int32 
dtypes: int32(1), object(1)
memory usage: 83.3+ KB


In [ ]:
test_data = test_data.astype({'label':'int32'})
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   problem  282 non-null    object
 1   label    282 non-null    int32 
dtypes: int32(1), object(1)
memory usage: 3.4+ KB


숫자형태의 카테고리로 인코딩을 수행한다.   
-> 그래야 loss가 잘 계산됨

In [ ]:
# label encoding 
from sklearn.preprocessing import LabelEncoder 

label_encoder = LabelEncoder() 
label_encoder.fit(data['class'])
num_labels = len(label_encoder.classes_) 

data['encoded_label'] = np.asarray(label_encoder.transform(data['class']), dtype=np.int32)
data.head() 

,problem,class,encoded_label
0,한 변의 길이가 24cm인 정육각형과 둘레가 같은 정팔각형이 있습니다. 이 정팔각형...,7,7
1,윤아는 부추전을 똑같이 8조각으로 나누어 한 조각을 먹었습니다. 윤미는 같은 크기의...,0,0
2,"화단 주변에 한 변이 12m인 정팔각형 모양의 울타리를 두른다면, 울타리는 모두 몇...",7,7
3,"6장의 숫자 카드 0, 9, 8, 7, 2, 1가 있습니다. 이를, 한 번씩 사용하...",2,2
4,0.26 x 0.8을 계산해 주세요.,0,0


In [ ]:
problem = data.problem.to_list()
classes = data['encoded_label'].to_list() 

In [ ]:
problem[0], classes[:2]

('한 변의 길이가 24cm인 정육각형과 둘레가 같은 정팔각형이 있습니다. 이 정팔각형의 한 변의 길이는 몇 cm인지 구하시오.',
 [7, 0])

텍스트와 라벨을 따로 분리한다.

In [ ]:
from sklearn.model_selection import train_test_split 

train_prob, val_prob, train_class, val_class = train_test_split(problem,
                                                                classes, 
                                                                test_size=0.1, 
                                                                random_state=5)

In [ ]:
len(train_prob)

6386

# 2- 모델 및 토크나이저 불러오기
hugging face에서 monologg/koelectra-base-v3-discriminator 모델과   
Electra 토크나이저를 가져온다.

In [ ]:
from transformers import ElectraTokenizer

model_path = 'monologg/koelectra-base-v3-discriminator'

tokenizer = ElectraTokenizer.from_pretrained(model_path)

# 문제 토큰화
train_encodings = tokenizer(train_prob, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_prob, truncation=True, padding=True, max_length=256) 

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

토큰화 된 데이터셋을 Tensorflow의 Dataset object로 변환

In [ ]:
import tensorflow as tf  

# train set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_class
))

# validation set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings), 
    val_class
))

In [ ]:
train_dataset

<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(117,), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(117,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(117,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

# 3- Koelectra fine-tuning 하기
Text Classification 이 목적이므로 TFElectraForSequenceClassification 클래스를 활용한다.

In [ ]:
from transformers import TFElectraForSequenceClassification

num_labels = len(label_encoder.classes_)

model = TFElectraForSequenceClassification.from_pretrained(model_path,
                                                           num_labels = num_labels,
                                                           from_pt=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer = optimizer, )#loss=model.compute_loss, metrics=['accuracy'],)

Downloading:   0%|          | 0.00/431M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraForSequenceClassification: ['electra.embeddings.position_ids', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing TFElectraForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFElectraForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dens

In [ ]:
#!pip install pyyaml h5py

모델 학습시키기

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os 

callback_es = EarlyStopping(
    monitor = 'val_loss',
    patience = 2
)

mc = ModelCheckpoint('training_1/cp.ckpt', monitor='val_loss', mode='min', verbose = 1,
                     save_best_only=True)


model.fit(
    train_dataset.shuffle(2538).batch(16), epochs = 10, batch_size = 16,
    validation_data = val_dataset.shuffle(1000).batch(16),
    callbacks = [callback_es, mc]
)


Epoch 1/10
400/400 [==============================] - ETA: 0s - loss: 0.5789
Epoch 1: val_loss improved from inf to 0.19034, saving model to training_1/cp.ckpt


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


400/400 [==============================] - 183s 395ms/step - loss: 0.5789 - val_loss: 0.1903
Epoch 2/10
400/400 [==============================] - ETA: 0s - loss: 0.2410
Epoch 2: val_loss did not improve from 0.19034
400/400 [==============================] - 108s 271ms/step - loss: 0.2410 - val_loss: 0.2077
Epoch 3/10
400/400 [==============================] - ETA: 0s - loss: 0.1917
Epoch 3: val_loss improved from 0.19034 to 0.09880, saving model to training_1/cp.ckpt


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


...

INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


400/400 [==============================] - 148s 371ms/step - loss: 0.1115 - val_loss: 0.0800
Epoch 5/10
400/400 [==============================] - ETA: 0s - loss: 0.0514
Epoch 5: val_loss improved from 0.07997 to 0.06952, saving model to training_1/cp.ckpt


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


400/400 [==============================] - 151s 378ms/step - loss: 0.0514 - val_loss: 0.0695
Epoch 6/10
400/400 [==============================] - ETA: 0s - loss: 0.0604
Epoch 6: val_loss did not improve from 0.06952
400/400 [==============================] - 108s 271ms/step - loss: 0.0604 - val_loss: 0.0821
Epoch 7/10
400/400 [==============================] - ETA: 0s - loss: 0.0459
Epoch 7: val_loss did not improve from 0.06952
400/400 [==============================] - 109s 272ms/step - loss: 0.0459 - val_loss: 0.0780


In [ ]:
#dir(model)

In [ ]:
train_dataset.element_spec[0]['input_ids']

TensorSpec(shape=(117,), dtype=tf.int32, name=None)

원래는 아래의 코드에서 주석 처리된 부분을 실행해서, 인코딩 된 라벨들을 원상복귀시켜야 하는데,   
어찌된 일인지 계속 에러가 나서 실행하지 않았다.  

이 부분은 아래에서 다시 처리할 예정이다.

In [ ]:
import re

id2labels = model.config.id2label
#print(id2labels)
#model.config.id2label = {id: label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] for id, label in id2labels.items()}
print(id2labels)

label2ids = model.config.label2id  
#model.config.label2id = {label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] :id for id, label in id2labels.items()}
print(label2ids)

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3', 4: 'LABEL_4', 5: 'LABEL_5', 6: 'LABEL_6', 7: 'LABEL_7'}
{'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2, 'LABEL_3': 3, 'LABEL_4': 4, 'LABEL_5': 5, 'LABEL_6': 6, 'LABEL_7': 7}


모델 저장하기

In [ ]:
model.save_pretrained('finetuned_koelectra.h5')
tokenizer.save_pretrained('finetuned_koelectra')

('finetuned_koelectra/tokenizer_config.json',
 'finetuned_koelectra/special_tokens_map.json',
 'finetuned_koelectra/vocab.txt',
 'finetuned_koelectra/added_tokens.json')

# 4- 저장된 모델 불러와서 클래스 예측하기

In [ ]:
# 위에서 분리한 test_data를 가져온다
test_data.head() 

,problem,label
0,진호가 가진 줄자의 길이는 6m 15 cm 이고 두은이가 가진 줄자의 길이는 0.2...,0
1,농구공 18개를 보관함 3개에 똑같이 나누어 담으려고 합니다. 보관함 1개에 농구공...,0
2,승준이는 우유를 매일 0.7L씩 마십니다. 승준이가 5일 동안 마실 우유를 준비하려...,0
3,원 안에 마름모의 넓이가 90cm²일 때 원의 반지름은 몇 cm일까요?,7
4,"가로가 45센티미터, 세로가 50센티미터인 직사각형 모양의 종이를 크기가 같은 정사...",7


In [ ]:
# 위에서 저장한 모델 및 토크나이저 가져오기
from transformers import TextClassificationPipeline 

loaded_tokenizer = ElectraTokenizer.from_pretrained('finetuned_koelectra')
loaded_model = TFElectraForSequenceClassification.from_pretrained('finetuned_koelectra.h5')

text_classifier = TextClassificationPipeline(
    tokenizer = loaded_tokenizer,
    model=loaded_model,
    framework='tf',
    return_all_scores=True
)


All model checkpoint layers were used when initializing TFElectraForSequenceClassification.

All the layers of TFElectraForSequenceClassification were initialized from the model checkpoint at finetuned_koelectra.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraForSequenceClassification for predictions without further training.


from_pretrained() 메소드에 저장된 모델 디렉토리를 넣어 fine-tuning한 model 및 tokenizer를 가져올 수 있다.  

transformers의 Pipelines 클래스를 사용하면 특정 task에 대한 inference를 수행할 수 있다.

In [ ]:
# 클래스 예측하기
predicted_label_list = [] 
predicted_score_list = [] 

for text in test_data['problem']: 
  #predict 
  preds_list = text_classifier(text)[0]

  sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
  
  predicted_label_list.append(sorted_preds_list[0]['label']) 
  predicted_score_list.append(sorted_preds_list[0]['score'])

결과가 어떻게 나오는지 확인

In [ ]:
preds_list

[{'label': 'LABEL_0', 'score': 0.0009193186997435987},
 {'label': 'LABEL_1', 'score': 2.5290102712460794e-05},
 {'label': 'LABEL_2', 'score': 8.571946818847209e-05},
 {'label': 'LABEL_3', 'score': 9.29472953430377e-05},
 {'label': 'LABEL_4', 'score': 0.00011254605487920344},
 {'label': 'LABEL_5', 'score': 0.00010870018741115928},
 {'label': 'LABEL_6', 'score': 0.0002488807949703187},
 {'label': 'LABEL_7', 'score': 0.9984065890312195}]

In [ ]:
preds_list[0].items()

dict_items([('label', 'LABEL_0'), ('score', 0.0009193186997435987)])

In [ ]:
len(predicted_label_list)
predicted_label_list[:5]

['LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_7', 'LABEL_7']

In [ ]:
predicted_score_list[:5]

[0.9990127086639404,
 0.9992882609367371,
 0.9972692131996155,
 0.9984121322631836,
 0.9981290698051453]

위에서 인코딩한 라벨을 처리해준다.   
LABEL_을 빼고 1을 더해 원래 class 값으로 저장한다.

In [ ]:
predicted_label_list_2 = []
for i in range(len(predicted_label_list)):
  predicted_label_list_2.append(re.sub('LABEL_', '', predicted_label_list[i]))
  predicted_label_list_2= list(map(int, predicted_label_list_2))
  #predicted_label_list_2[i] += 1
predicted_label_list_2[:5]

[0, 0, 0, 7, 7]

In [ ]:
len(sorted_preds_list)

8

test_data에 예측한 클래스와 확률값을 추가하여 확인한다.

In [ ]:
test_data['pred'] = predicted_label_list_2 
test_data['score'] = predicted_score_list
test_data.head()

,problem,label,pred,score
0,진호가 가진 줄자의 길이는 6m 15 cm 이고 두은이가 가진 줄자의 길이는 0.2...,0,0,0.999013
1,농구공 18개를 보관함 3개에 똑같이 나누어 담으려고 합니다. 보관함 1개에 농구공...,0,0,0.999288
2,승준이는 우유를 매일 0.7L씩 마십니다. 승준이가 5일 동안 마실 우유를 준비하려...,0,0,0.997269
3,원 안에 마름모의 넓이가 90cm²일 때 원의 반지름은 몇 cm일까요?,7,7,0.998412
4,"가로가 45센티미터, 세로가 50센티미터인 직사각형 모양의 종이를 크기가 같은 정사...",7,7,0.998129


# 5- 평가하기

In [ ]:
from sklearn.metrics import classification_report 

print(classification_report(y_true=test_data['label'], y_pred=test_data['pred']))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97       113
           1       1.00      1.00      1.00         2
           2       0.68      0.88      0.77        17
           3       0.77      0.65      0.71        26
           4       0.75      0.86      0.80        14
           5       1.00      0.87      0.93        15
           6       0.86      0.86      0.86        14
           7       0.99      0.99      0.99        81

    accuracy                           0.92       282
   macro avg       0.88      0.88      0.88       282
weighted avg       0.93      0.92      0.92       282



- 데이터 불균형을 해결한 후 정확도는 92%정도 나온다.
